# Your mission

You started working on the Ecowatt project at RTE. In order to avoid possible shortage, one must plan for peaks in national electricity. You manager Mark is going on holidays for a week. You will be sole responsible for forecasting the weekly demand, while he is absent.

In order to prevent electricity shortage, you must accurately forecast the demand 7 days ahead, on an hourly basis.

Your mission is to train an accurate predictive model with the lowest root mean squared error (RMSE). Mark is a very technical guy, he likes to understand all technical details and would like you to compare the performances of classical models and neural-net based models.


Your **target variable** is the consommation_totale

**Data source** : https://data.enedis.fr/pages/accueil/

# Import

In [37]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
drive.mount('/content/gdrive')
if os.getcwd() != "/content/gdrive/MyDrive/Colab Notebooks/EI_TS_CS":
  os.chdir("/content/gdrive/MyDrive/Colab Notebooks/EI_TS_CS")

In [38]:
%run ./utils.ipynb

In [39]:
FILE_PATH = "data/bilan-electrique.csv"
TARGET = "consommation_totale"

## Prepare the data

Define here the range of your train/test split

In [40]:
df = read_data(FILE_PATH)

X_train = df[-1000:-100]
X_test = df[-100:]

# Modeling with ARIMA
In this section, you are to perform some classical modelings, the suggested method here is ARIMA, but you can try other models such as ARMA, ARIMAX, SARIMAX...

## Modeling
The following code allows ARIMA modeling with one combination of (p,d,q).

In [41]:
parameters = (2,1,1)
errors, predictions = evaluate_arima_model(
    X_train[TARGET],
    X_test[TARGET],
    parameters
    )
errors

9.249299221466796e+17

## Search for the best ARIMA model
We use grid search to search for the best ARIMA parameters that gives the lowest error. This follows the Box-Jenkins methology.

In [42]:
best_cfg, best_score = arima_grid_search(X_train[TARGET],
                                            X_test[TARGET],
                                            range(1,3),range(0,3),range(0,3))

ARIMA(1,0,0) RMSE=42273464102392512512.000
ARIMA(1,0,1) RMSE=178145348234236559360.000


/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/py

ARIMA(1,0,2) RMSE=224105685470580521828352.000
ARIMA(1,1,0) RMSE=1221278597940926976.000
ARIMA(1,1,1) RMSE=931176290890376448.000
ARIMA(1,1,2) RMSE=910433750458464000.000
ARIMA(1,2,0) RMSE=1397453910008929280.000
ARIMA(1,2,1) RMSE=1197202546814268672.000
ARIMA(1,2,2) RMSE=1143371913313136768.000
ARIMA(2,0,0) RMSE=118813861289912532992.000
ARIMA(2,0,1) RMSE=4326850427720821760.000


/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/nathanbrites/code/EI_TS_CS/venv/lib/py

ARIMA(2,0,2) RMSE=4575994011924638466048.000
ARIMA(2,1,0) RMSE=1078356819737757952.000
ARIMA(2,1,1) RMSE=924929922146679552.000


/Users/nathanbrites/code/EI_TS_CS/venv/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.



ARIMA(2,1,2) RMSE=910989080083503360.000
ARIMA(2,2,0) RMSE=1179195714780165888.000
ARIMA(2,2,1) RMSE=1195370010630024192.000
ARIMA(2,2,2) RMSE=1143944657300845824.000
Best ARIMA(1, 1, 2) MSE=910433750458464000.000


In [43]:
print(best_cfg, best_score)

(1, 1, 2) 9.10433750458464e+17


## Visualization
To have a better view on the difference between true and predict values, we visualize them by plotting both the signals.

In [44]:
# prepare the dataset for plotting
predict_date = df["horodate"]
df_predict = pd.DataFrame(zip(predict_date,
                              predictions, X_test[TARGET].values),
                          columns=["date", "predict", "true"])

In [45]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["predict"], name="predict"))
fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["true"], name="true"))

fig.update_layout(title="Predictions vs true values")

# Modeling with other models

Try other other models : random forest, xgboost ...